## Load 2020 WIDE-formatted ESG data (Shell)

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### Initially developed using the Royal Dutch Shell plc Sustainability Report 2020 report (Many Sheets)

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Load Credentials

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
import re
import pandas as pd
import numpy as np

import openpyxl
from openpyxl import load_workbook
from itertools import islice

import pint
import pint_pandas

from osc_ingest_trino import *
import pyarrow as pa
import pyarrow.parquet as pq
import json
import io
import uuid

For spreadsheets in WIDE format, pre-process the spreadsheet as a workbook, cascading label data into 3rd-normal form row and column metadata

* var_col is the label of the variable being measured (whose specificity (like CO2, CH4, NOx, etc) often affects units)
* units_col is the column where units are stated
* val_col is the column where the first value is quantitatively reported

We add:
* notes_col (source worksheet-specific; could act as a kind of source table metadata)
* topic_col (sheet-level category; if we wanted large tables, they could be named by topic)
* category_col (to which row-level data rolls up; if we wanted small tables, they could be named by topic:category)
* segment_col (the dimension by which row-level data is segmented)

Some spreadsheets use color to express a multi-level category hierarchy (such as Energy Consumption>>Business Use>>Fuel Type).  We concatenate the categories from left to right as the category for our purposes, except we split off the rightmost subcategory as the segmentation.

Based on all of the above, we don't really have table-level metadata other than notes attached to sheets and generic column information.  An argument could be made that we need to allocate specifier columns for additional data we want to split out from our variables.  That could look like:

* spec1_col
* spec2_col

etc

In [ ]:
# Magic knowledge
var_col = 1

ingest_columns = [ 'Variable', 'Notes', 'Topic', 'Category', 'Segmentation' ]
ingest_dict = dict((j,i) for i,j in enumerate(ingest_columns[1:], start=var_col+1))
notes_col = ingest_dict['Notes']
topic_col = ingest_dict['Topic']
category_col = ingest_dict['Category']
segmentation_col = ingest_dict['Segmentation']

# Magic knowledge
units_col = segmentation_col+1   # units_col starts as var_col+1
val_col = segmentation_col+2     # val_col starts as var_col+2
topic_row = 3
header_row = 5

In [ ]:
# We need all this complexity to find fill colors, if any, for discerning additional category hierarchies (which we flatten by concatenation).

from colorsys import rgb_to_hls, hls_to_rgb
# From: https://stackoverflow.com/questions/58429823/getting-excel-cell-background-themed-color-as-hex-with-openpyxl/58443509#58443509
#   which refers to: https://pastebin.com/B2nGEGX2 (October 2020)
#       Updated to use list(elem) instead of the deprecated elem.getchildren() method
#       which has now been removed completely from Python 3.9 onwards.
#

#https://bitbucket.org/openpyxl/openpyxl/issues/987/add-utility-functions-for-colors-to-help

RGBMAX = 0xff  # Corresponds to 255
HLSMAX = 240  # MS excel's tint function expects that HLS is base 240. see:
# https://social.msdn.microsoft.com/Forums/en-US/e9d8c136-6d62-4098-9b1b-dac786149f43/excel-color-tint-algorithm-incorrect?forum=os_binaryfile#d3c2ac95-52e0-476b-86f1-e2a697f24969

def rgb_to_ms_hls(red, green=None, blue=None):
    """Converts rgb values in range (0,1) or a hex string of the form '[#aa]rrggbb' to HLSMAX based HLS, (alpha values are ignored)"""
    if green is None:
        if isinstance(red, str):
            if len(red) > 6:
                red = red[-6:]  # Ignore preceding '#' and alpha values
            blue = int(red[4:], 16) / RGBMAX
            green = int(red[2:4], 16) / RGBMAX
            red = int(red[0:2], 16) / RGBMAX
        else:
            red, green, blue = red
    h, l, s = rgb_to_hls(red, green, blue)
    return (int(round(h * HLSMAX)), int(round(l * HLSMAX)), int(round(s * HLSMAX)))

def ms_hls_to_rgb(hue, lightness=None, saturation=None):
    """Converts HLSMAX based HLS values to rgb values in the range (0,1)"""
    if lightness is None:
        hue, lightness, saturation = hue
    return hls_to_rgb(hue / HLSMAX, lightness / HLSMAX, saturation / HLSMAX)

def rgb_to_hex(red, green=None, blue=None):
    """Converts (0,1) based RGB values to a hex string 'rrggbb'"""
    if green is None:
        red, green, blue = red
    return ('%02x%02x%02x' % (int(round(red * RGBMAX)), int(round(green * RGBMAX)), int(round(blue * RGBMAX)))).upper()


def get_theme_colors(wb):
    """Gets theme colors from the workbook"""
    # see: https://groups.google.com/forum/#!topic/openpyxl-users/I0k3TfqNLrc
    from openpyxl.xml.functions import QName, fromstring
    xlmns = 'http://schemas.openxmlformats.org/drawingml/2006/main'
    root = fromstring(wb.loaded_theme)
    themeEl = root.find(QName(xlmns, 'themeElements').text)
    colorSchemes = themeEl.findall(QName(xlmns, 'clrScheme').text)
    firstColorScheme = colorSchemes[0]

    colors = []

    for c in ['lt1', 'dk1', 'lt2', 'dk2', 'accent1', 'accent2', 'accent3', 'accent4', 'accent5', 'accent6']:
        accent = firstColorScheme.find(QName(xlmns, c).text)
        for i in list(accent): # walk all child nodes, rather than assuming [0]
            if 'window' in i.attrib['val']:
                colors.append(i.attrib['lastClr'])
            else:
                colors.append(i.attrib['val'])

    return colors

def tint_luminance(tint, lum):
    """Tints a HLSMAX based luminance"""
    # See: http://ciintelligence.blogspot.co.uk/2012/02/converting-excel-theme-color-and-tint.html
    if tint < 0:
        return int(round(lum * (1.0 + tint)))
    else:
        return int(round(lum * (1.0 - tint) + (HLSMAX - HLSMAX * (1.0 - tint))))

def theme_and_tint_to_rgb(wb, theme, tint):
    """Given a workbook, a theme number and a tint return a hex based rgb"""
    rgb = get_theme_colors(wb)[theme]
    h, l, s = rgb_to_ms_hls(rgb)
    return rgb_to_hex(ms_hls_to_rgb(h, tint_luminance(tint, l), s))

# ??? The header row color is going to be spreadsheet-specific.  This is what DPDHL gives us.

def find_header_row(wb, ws):
    # If we haven't found the header by max_row-1, we'll never find it...
    for row in range(1, ws.max_row):
        cell = ws.cell(row,1)
        if cell.fill.fgColor.type == 'rgb':
            if cell.fill.fgColor.rgb == 'FFBF00':
                return row
            continue
        theme = cell.fill.start_color.theme
        tint = cell.fill.start_color.tint
        color = theme_and_tint_to_rgb(wb, theme, tint)
        if color=='FFBF00':
            return row
    print('No header found')
    return -1

In [ ]:
# We pre-process the structure of the worksheet so that it can be trivially loaded into a dataframe for further reshaping.

# Stash notes for each worksheet here.  These are *per worksheet*
# ??? In the case of DPDHL, there's a Comment field we don't track, which means we miss a stated target
ws_notes = {}
    

def preprocess(wb, ws):
    
    # Intended for Shell notes
    def save_ws_notes(ws, note):
        global ws_notes
        
        if ws.title not in ws_notes:
            ws_notes[ws.title] = {}
        note_label, note_text = note.split(' ', 1)
        ws_notes[ws.title][note_label] = note_text.strip()
    
    # Intended for DPDHL notes
    def save_ws_notes2(ws, note):
        global ws_notes
        
        if ws.title not in ws_notes:
            ws_notes[ws.title] = {}
        notes = re.split(r' (\d+)\)\s+')
        print('NOTES')
        print(notes)
        print('END NOTES')
        for i in range(len(notes)/2):
            ws_notes[ws.title][notes[2*i]] = notes[2*i+1].strip()
    
    scope1_gases = ['CO2', 'CH4', 'N2O', 'HFC', 'SF6', 'PFC', 'NF3', 'CO2e', 'NOx', 'SO2', 'PM10']
    scope1_regex = re.compile('(' + ')|('.join(scope1_gases) + ')', flags=re.I)
    
    scope3_dict = { 'Purchased Goods and Services':1,
                    'Capital Goods':2,
                    'Fuel and Energy Related Activities':3,
                    'Fuel and Energy Related Activities (Market-Based)':3,
                    'Fuel and Energy Related Activities (Location-Based)':3,
                    'Upstream Transportation and Distribution':4,
                    'Transportation services':4,                # DPDHL
                    'Fuel- and energy-related activities':4,    # DPDHL
                    'Waste Generated in Operations (Large office campuses)':5,
                    'Business Travel':6,
                    'Employee Commuting':7,
                    'Upstream Leased Assets':8,
                    'Downstream Transportation and Distribution':9,
                    'Processing of Sold Products':10,
                    'Use of Sold Products':11,
                    'End of Life Treatment of Sold Products':12,
                    'Downstream Leads Assets':13,
                    'Franchises':14,
                    'Investments':15 }

    def normalize_scope3(s3):
        # Later we should normalize against the scope3_dict
        return s3
    
    # The role of this function is to capture and distrbute data attributes that can be inferred/applied to subsequent rows
    def split_header(c):
        notes = ''
        
        # Deal with None
        if c.value:
            h = str(c.value)
        else:
            h = ''
        
        # Look for superscripts that openpxyl has parsed and disposed of before we can see them
        m = re.search(r'^20\d\d', h)
        if m:
            notes = h[4:]
            return notes, m.group(0), ''
        
        # Don't let 'Scope 1' look like a note
        m = re.search(r'[^ ](\d+)$', h)
        if m:
            notes = m.group(1)
            h = h[0:m.start(1)]
        else:
            m = re.search(r'\[.*\]', h)
            if m:
                notes = m.group(0)
                h = h.replace(notes,'').strip()
        
        # If the variable expresses a segmentation, pass that back accordingly
        for x in [ ' per ', ' by ', ' of ' ]:
            sub_h_arr = h.split(x, 1)
            if len(sub_h_arr)>1:
                return notes, sub_h_arr[0], sub_h_arr[1]
        
        # Treat X (Y) as 'Category X Segmentation Y'
        m = re.search(r'^(.*) \((.*)\)', h)
        if m:
            return notes, m.group(1), m.group(2)
        return notes, h, ''
    
    # Convert reported units to things standard in `pint`
    unit_dict = { 'trillion (10^12) MJ':'PJ', 'million MWh':'TWh', 'million tonnes CO2e': 'Mt CO2e',
                 'million tonnes': 'Mt', 'thousand tonnes': 'kilot', 'tonnes': 't', 'tBtu':'TBtu',
                  'm t CO2e':'Mt CO2e', 'm liter':'M liter', 'Grams per € revenue': 'Grams / EUR' }
    def normalize_units(u, g):
        if g in u:
            g = ''
        if '/' in u:
            u1, u2 = u.split('/', 1)
            if g in u2:
                g1 = ''
                g2 = g
            else:
                g1 = g
                g2 = ''
            return ' / '.join([normalize_units(u1, g1), normalize_units(u2, g2)])
        u = u.strip()
        if u in unit_dict:
            return normalize_units(unit_dict[u], g)
        if g:
            return ' '.join([u, g])
        return u
    
    def crop_sheet(ws):
        # Frist, set max_row/max_column based on actually active cells, not cells with random spaces or empty strings
        this_max_row = 1
        this_max_col = 1
        for row in range(1,ws.max_row+1):
            for col in range(1,ws.max_column+1):
                cell = ws.cell(row,col)
                if cell.value==None:
                    continue
                if type(cell.value)==str and cell.value.strip()=='':
                    cell.value = None
                    continue
                if row > this_max_row:
                    this_max_row = row
                if col > this_max_col:
                    this_max_col = col
        ws.delete_rows(this_max_row+1,ws.max_row)
        ws.delete_cols(this_max_col+1,ws.max_column)
    
    crop_sheet(ws)

    # Make space for TOPIC : CATEGORY : SEGMENTATION triple.
    # This triple could very well become an index into a data framework (such as SASB, TCFD, etc)
    ws.insert_cols(var_col+1,amount=segmentation_col-var_col)
    if True:
        # All Shell headers start in the smae place
        header_row = 5
    else:
        header_row = find_header_row (wb, ws)

    for col in range(var_col, units_col):
        ws.cell(header_row, col).value = ingest_columns[col-var_col]
    # Find notes in header line (such as a callout that measurement systems changed in a particular year)
    for col in range(units_col, ws.max_column+1):
        notes, main_text, segmentation = split_header(ws.cell(header_row, col))
        if notes:
            # main_text has the note removed from it
            ws.cell(header_row, col).value = main_text
            if segmentation:
                print('found note, but lost this: ' + segmentation)
    
    topic = ws.cell(topic_row,var_col).value
    
    notes = ''
    category = ''
    # DPDHL expresses a 3-level cat hierarchy via bold plus dark green, light green, and white backgrounds.  Also Grays for "Other Env. Data"
    cat_color_dict = { None:0, 'FF00B050':0, 'E2F0D9':1, '00000000':2, 'D0CECE':0, 'E7E6E6':0 }
    categories = ['', '', '']
    segmentation = ''
    scope1_gas = ''
    
    # Make the inferences, filling out TOPIC : CATEGORY : SEGMENTATION, as well as inferring/adjusting UNITS
    for row in range(header_row+1, ws.max_row+1):
        cell = ws.cell(row, var_col)
        ws.cell(row, topic_col).value = topic
        if cell.value==None:
            continue
        # Find either bracketed note or note that begins with possible superscript
        if cell.value[0]=='[':
            save_ws_notes(ws, cell.value)
            continue
        elif re.search(r'^[^(]*\d[)]', str(cell.value)):
            save_ws_notes2(ws, cell.value)
        
        # *BOLD* text indicates we have a header to parse
        if cell.font.b:
            if True:
                # Shell doesn't use colors
                cat_color = None
            elif cell.fill.fgColor.type == 'rgb':
                cat_color = format(cell.fill.fgColor.rgb)
            else:
                theme = cell.fill.start_color.theme
                tint = cell.fill.start_color.tint
                cat_color = theme_and_tint_to_rgb(wb, theme, tint)

            notes, category, segmentation = split_header(cell)
            categories[cat_color_dict[cat_color]] = re.sub(r' total\s?', '', category)
            for i in range(cat_color_dict[cat_color]+1, len(categories)):
                categories[i] = ''
            category = ':'.join([c for c in categories[0:2] if c])
            if re.search(r'Scope\s*3', category, flags=re.I):
                m = re.search(r'Scope\s+3 (emissions )(by.*categor((y)|(ies)))?', category, flags=re.I)
                category = 'Scope 3 emissions'
                segmentation = 'GHG Categories'
            else:
                if categories[2]:
                    if segmentation:
                        print('cat[2] = {}; segmentation = {}'.format(categories[2], segmentation))
                    segmentation = categories[2]
            
            # If we have no units, borrow from following row
            if ws.cell(row, units_col).value==None:
                ws.cell(row, units_col).value = ws.cell(row+1, units_col).value
            # If there is no disclosure here, move on with the notes/category/segmentation we've captured
            if ws.cell(row, val_col).value==None:
                continue
            
            if category == 'Scope 3 emissions' and segmentation == 'GHG Categories':
                ws.cell(row, var_col).value = normalize_scope3 (ws.cell(row, var_col).value)
            
        # Try to get units and category from variable description.  In the Shell case, which mostly fills in units,
        # we try vary hard to fill in empty units with parenthetical expressions.  In DPDHL case, which mostly does not,
        # we want to infer more from previous rows or cascade a defined unit from the current row to subsequent rows
        if ws.cell(row, units_col).value==None:
            maybe_notes, maybe_category, maybe_var_units = split_header(cell)
            if type(ws.cell(row, units_col)) == openpyxl.cell.cell.Cell:
                # Don't try to update a MergedCell
                maybe_var_units = re.sub(r'\((.*)\)', r'\1', maybe_var_units)
                if maybe_var_units in unit_dict:
                    units = maybe_var_units
                ws.cell(row, units_col).value = units
            else:
                notes, category = maybe_notes, maybe_category
                if maybe_var_units:
                    units = maybe_var_units
                print('Merged cell: {} {}'.format(row, units_col))
        # Now fill the empty columns we created with the metadata we have inferred
        ws.cell(row, notes_col).value = notes
        ws.cell(row, category_col).value = category
        ws.cell(row, segmentation_col).value = segmentation
        if 'emissions' in category.lower() and not re.search('CO2e', ws.cell(row, units_col).value, re.I):
            m = re.search(scope1_regex, str(ws.cell(row, var_col).value))
            if m:
                scope1_gas = m.group(0)
            # else it carries forward
        else:
            scope1_gas = ''
        if ws.cell(row, units_col).value!=None:
            units = normalize_units(ws.cell(row, units_col).value, scope1_gas)
            ws.cell(row, units_col).value = units

In [ ]:
# With a nicely formatted workbook, do the rest of our work (including writing to Trino) using dataframes

# IPIECA, SASB, and GRI columns all feed metadata

header_row_list = [ -1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5 ]

def shell_ws_to_df(wb, i):
    data = islice(wb.worksheets[i].values, header_row_list[i]-1, None)
    cols = list(next(data))
    data = list(data)
    # idx = [r[0] for r in data]
    # data = (islice(r, 0, None) for r in data)
    cols[units_col-1] = 'Unit'            # Already set by Shell
    df = pd.DataFrame(data, columns=cols) # we don't pass in an index here

    # df.dropna(subset=['Unit'], inplace=True)
    df.replace(to_replace='^n/[acd]$', value='', regex=True, inplace=True)
    
    # Remove null columns
    df = df[[c for c in df.columns if c!= None]]
    # Change numerical years to strings to make pandas indexing behave
    df.columns = [str(c) for c in df.columns]
    # Drop completely empty rows
    df.dropna(how='all', axis=0, inplace=True)
    return df

Write out polymorphic dataframe in LONG format.  This follows tidy data model, with one variable observation per row.  
Polymorphic means that Units/dimensions of each row are specified, but not necessarily the same row to row.  
Aggregation functions must be careful that selection criteria does not mix up incompatible unit types and/or observation variables.

In [ ]:
filename = os.environ.get('PWD', '/opt/app-root/src') + '/osc-ingest-shell/data/raw/greenhouse-gas-and-energy-data-shell-sr20.xlsx'
wb = load_workbook(filename)

# For a label like "Scope 1 emissions by country" return ['', 'Scope 1 emissions', 'country']
# For a label like "Direct GHG emissions (Scope 1) [A] [B] [C] [D]" return ['[A] [B] [C] [D]', 'Direct GHG emissions (Scope 1) ', '']

with pd.ExcelWriter(os.environ.get('PWD', '/opt/app-root/src') + '/osc-ingest-shell/data/processed/Shell_2020_LONG.xlsx') as writer_long:
    with pd.ExcelWriter(os.environ.get('PWD', '/opt/app-root/src') + '/osc-ingest-shell/data/processed/Shell_2020_WIDE.xlsx') as writer_wide:
        # We skip the first sheet as it's just a table of contents
        for i in range(1, len(wb.worksheets)):
            ws = wb.worksheets[i]
            ws_notes = {}
            preprocess(wb, ws)
            # What to do with ws_notes???
            df = shell_ws_to_df(wb, i)
            df.replace('',pd.NA,inplace=True)
            melted_df = pd.melt(df, id_vars=ingest_columns+['Unit'], var_name='Year', value_name='Value', value_vars=['2016', '2017', '2018', '2019', '2020'])
            melted_df.dropna(subset=['Value'],inplace=True)
            melted_df = melted_df.astype({'Year': 'int'})
            # This writes out LONG data with TOPIC as SHEET_NAME.  Later we'll create a truly long table with TOPIC restored as a column
            melted_df.loc[:, melted_df.columns != 'Topic'].to_excel(writer_long, index=False, sheet_name=df.iloc[0]['Topic'][0:30])

            print(ws.title)
            columns = ['Variable', 'Unit']
            # We need these columns to reshape our data
            for extra_col in ['Notes', 'Category', 'Segmentation']:
                if df[extra_col].notna().any():
                    columns.append(extra_col)
            # In the case of Shell, we have only one topic per sheet, so can transform melted_df directly
            pf = melted_df.pivot(index=['Year', 'Topic'], columns=columns, values=['Value'])
            pf = pf.droplevel('Topic')
            # Once reshaped, the extra columns actually appear as multi-level indexes.  Drop them from also behaving like values
            pf[[c for c in columns if c not in ['Variable', 'Unit']]] = pd.NA
            pf.dropna(how='all', axis=1, inplace=True)
            pf.to_excel(writer_wide, sheet_name=df.iloc[0]['Topic'][0:30])

Make the workbook more legible to those reading it

In [ ]:
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter
long_filename = os.environ.get('PWD', '/opt/app-root/src') + '/osc-ingest-shell/data/processed/Shell_2020_LONG.xlsx'

wb = load_workbook(long_filename)

# We are now working with our own workbook, which doesn't have a zero-index sheet to ignore
for ws in wb.worksheets:
    dim_holder = DimensionHolder(worksheet=ws)
    for col in range(ws.min_column, ws.max_column + 1):
        if get_column_letter(col)=='A':
            width = 40
        elif get_column_letter(col) in ['B', 'E']:
            width = 15
        elif get_column_letter(col) in ['C', 'D']:
            width = 25
        else:
            width = 10
        dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=width)
    ws.column_dimensions = dim_holder

wb.save(long_filename)
wb.close()

Write out dataframe in WIDE format.  This data is technically tiday, with one multi-dimensional observation per row.  Units/dimensions are consistent on a per-column basis, making it easy to aggregate column-based data.

Make the workbook more legible to those reading it

In [ ]:
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Font

def as_text(value):
    if value is None:
        return ""
    return str(value)

wide_filename = os.environ.get('PWD', '/opt/app-root/src') + '/osc-ingest-shell/data/processed/Shell_2020_WIDE.xlsx'

wb = load_workbook(wide_filename)

for ws in wb.worksheets:
    dim_holder = DimensionHolder(worksheet=ws)
    for col in range(ws.min_column, ws.max_column + 1):
        cell = ws.cell(2, col)
        cell.alignment = Alignment(wrap_text=True,vertical='top') 
        dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=max(10,1+len(as_text(cell.value))/3))
    ws.column_dimensions = dim_holder

wb.save(wide_filename)
wb.close()

### Time for a Pint!

See https://github.com/IAMconsortium/units/issues/9https://github.com/IAMconsortium/units/issues/9
and https://github.com/openscm/openscm-units/issues/31https://github.com/openscm/openscm-units/issues/31
and 

In [ ]:
import pandas as pd
import pint_pandas
from openscm_units import unit_registry

pint_pandas.PintType.ureg = unit_registry

one_co2 = unit_registry("CO2")
print(one_co2)

x = pd.DataFrame([[2.0,'Mt CO2']], columns=['Value', 'Unit'])
print(x)
x = x.astype({'Value': 'pint[Mt CO2]'})
print(x.Value.pint.to('t CO2'))

In [ ]:
PA_ = pint_pandas.PintArray

ureg = unit_registry
Q_ = ureg.Quantity

Note that pint[unit] must be used for the Series constuctor, whereas the PintArray constructor allows the unit string or object.

```
    df = pd.DataFrame({
        "length" : pd.Series([1.,2.], dtype="pint[m]"),
        "width" : PA_([2.,3.], dtype="pint[m]"),
        "distance" : PA_([2.,3.], dtype="m"),
        "height" : PA_([2.,3.], dtype=ureg.m),
        "depth" : PA_.from_1darray_quantity(Q_([2,3],ureg.m)),
    })
```

See https://pint.readthedocs.io/en/0.18/pint-pandas.html

In [ ]:
wb = load_workbook(long_filename)

from itertools import islice

def long_ws_to_df(ws):
    data = ws.values
    cols = next(data)
    data = list(data)
    # idx = [r[0] for r in data]
    # data = (islice(r, 1, None) for r in data)
    
    df = pd.DataFrame(data, columns=cols)

    # The original data has topic we construct.  It is removed when writing LONG data but can be restored from SHEET_NAME
    if 'Topic' not in df.columns:
        print('Restoring Topic ' + ws.title)
        df.insert(topic_col-1, 'Topic', ws.title)
    
    return df

shell_df = pd.concat([long_ws_to_df(ws) for ws in wb.worksheets])
    
len(shell_df)

In [ ]:
print(shell_df['Unit'].value_counts())
shell_df.Unit.unique()

Now create data in Trino

In [ ]:
import boto3

# Create an S3 client.  We will user later when we write out data and metadata
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_DEV_ENDPOINT'],
    aws_access_key_id=os.environ['S3_DEV_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_DEV_SECRET_KEY'],
)

In [ ]:
import trino

conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    verify=True,
)
cur = conn.cursor()

# Show available schemas to ensure trino connection is set correctly
cur.execute('show schemas in osc_datacommons_dev')
cur.fetchall()

In [ ]:
import datetime
# datetime.datetime.now()
# For now we used a fixed date so we don't fill things up needlessly
timestamp = "2008-09-03T20:56:35.450686Z"

In [ ]:
ingest_uuid = str(uuid.uuid4())

custom_meta_key_fields = 'metafields'
custom_meta_key = 'metaset'

schemaname = 'osc_corp_data'
cur.execute('create schema if not exists osc_datacommons_dev.' + schemaname)
cur.fetchall()

For osc_datacommons_dev, a trino pipeline is a parquet data stored in the S3_DEV_BUCKET
It is a 5-step process to get there from a pandas dataframe

In [ ]:
def create_trino_pipeline (s3, schemaname, tablename, timestamp, df, meta_fields, meta_content):
    global ingest_uuid
    global custom_meta_key_fields, custom_meta_key
    
    # First convert dataframe to pyarrow for type conversion and basic metadata
    table = pa.Table.from_pandas(enforce_sql_column_names(df))
    # Second, since pyarrow tables are immutable, create a new table with additional combined metadata
    if meta_fields or meta_content:
        meta_json_fields = json.dumps(meta_fields)
        meta_json = json.dumps(meta_content)
        existing_meta = table.schema.metadata
        combined_meta = {
            custom_meta_key_fields.encode(): meta_json_fields.encode(),
            custom_meta_key.encode(): meta_json.encode(),
            **existing_meta
        }
        table = table.replace_schema_metadata(combined_meta)
    # Third, convert table to parquet format (which cannot be written directly to s3)
    pq.write_table(table, '/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp))
    # df.to_parquet('/tmp/{sname}.{tname}.{uuid}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, index=False))
    # Fourth, put the parquet-ified data into our S3 bucket for trino.  We cannot compute parquet format directly to S3 but we can copy it once computed
    s3.upload_file(
        Bucket=os.environ['S3_DEV_BUCKET'],
        Key='trino/{sname}/{tname}/{uuid}/{timestamp}/{tname}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp),
        Filename='/tmp/{sname}.{tname}.{uuid}.{timestamp}.parquet'.format(sname=schemaname, tname=tablename, uuid=ingest_uuid, timestamp=timestamp)
    )
    # Finally, create the trino table backed by our parquet files enhanced by our metadata
    cur.execute('.'.join(['drop table if exists osc_datacommons_dev', schemaname, tablename]))
    print('dropping table: ' + tablename)
    cur.fetchall()
    
    schema = create_table_schema_pairs(df)

    tabledef = """create table if not exists osc_datacommons_dev.{sname}.{tname}(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{bucket}/trino/{sname}/{tname}/{uuid}/{timestamp}'
)""".format(schema=schema,bucket=os.environ['S3_DEV_BUCKET'],sname=schemaname,tname=tablename,uuid=ingest_uuid,timestamp=timestamp)
    print(tabledef)

    # tables created externally may not show up immediately in cloud-beaver
    cur.execute(tabledef)
    cur.fetchall()

### Write out Report with metadata

Create the actual metadata for the source.  In this case, it is osc_corp_data.

In [ ]:
custom_meta_content = {}
metadata_text = """Title: Shell GHG and Energy Report, 2020
Description: 
Version: 2020
Release Date: 
URI: https://reports.shell.com/sustainability-report/2020/our-performance-data/greenhouse-gas-and-energy-data.html
Copyright: 
License: 
Contact: 
Citation: """

for line in metadata_text.split('\n'):
    k, v = line.split(':', 1)
    k = sql_compliant_name(k)
    custom_meta_content[k] = v

custom_meta_content['abstract'] = """Abstract text"""
custom_meta_content['name'] = 'osc_corp_data'

Create the metadata for all the fields in all the tables

Create custom meta data and key

In [ ]:
shell_df

In [ ]:
tablename = 'shell_2020'
custom_meta_fields = {}
create_trino_pipeline (s3, schemaname, tablename, timestamp, shell_df, custom_meta_fields, custom_meta_content)

Restore data and metadata

In [ ]:
# Everything below here is speculative / in process of design

## Load metadata following an ingestion process into trino metadata store

### The schema is *metastore*, and the table names are *meta_schema*, *meta_table*, *meta_field*

In [ ]:
# Create metastore structure
metastore = {'catalog':'osc_datacommons_dev',
             'schema':'shell_2020',
             'table':tablename,
             'metadata':custom_meta_content,
             'uuid':ingest_uuid}
# Create DataFrame
df_meta = pd.DataFrame(metastore)
# Print the output
df_meta